# Black-Scholes model for option pricing

The Black-Scholes model is a mathematical model for option pricing used to calculate the theoretical price of European options, which give the owner the option to buy or sell an asset at a fixed price on a specific date in the future. The model assumes the options follow a lognormal distribution and are traded in a frictionless market where there are no transaction costs, no taxes and no restrictions on short selling.

The Black-Scholes model takes into account five different factors that affect the price an option:

* Strike price
* Current price of the underlying asset
* Time remaining until expiration of the asset
* Risk-free rate 
* Volatility of the underlying asset


### Model equations

Call option price:

$$C = S_t N(d_1) - Ke^{-r(T-t)}N(d_2)$$
Put option price:

$$P = Ke^{-r(T-t)}N(-d_2) - S_t N(-d_1)$$

Where:

$$d_1 = \frac{ln(\frac{S_t}{K}) + (r+\frac{\sigma^2}{2})(T-t)}{\sigma \sqrt{T-t}}$$

$$d_2 = d_1 - \sigma \sqrt{T-t}$$

## Import of the necessary libraries and modules

In [4]:
import numpy as np
from scipy.stats import norm

## Defining a function to calculate prices using the model

In [5]:
#Define parameters
S = 'Strike Price'
C = 'Current Price'
t = 'Time period to expiration'
r = 'Risk-free rate'
sigma = 'Volatility'

# Black scholes model price function
def option_price(S, C, t, r, sigma, option_type):
    d1 = (np.log(C/S) + (r + sigma**2/2)*t) / (sigma * np.sqrt(t)) # Expected return adjusted for the risk
    d2 = d1 - (sigma * np.sqrt(t)) # Takes into account the standard deviation of the returns
    
    print('d1: ', d1)
    print('d2: ', d2)
    if option_type == "Put":
        price = (S * np.exp(-r*t) * norm.cdf(-d2)) - (C * norm.cdf(-d1))
        print('Put option price: ', price)
    elif option_type == "Call":
        price = (C * norm.cdf(d1)) - (S * np.exp(-r*t) * norm.cdf(d2))
        print('Call option price: ', price)
    else:
        print('Invalid option type')

## Calling the function for different option pricing methods

In [6]:
#Run function
option_price(150, 110, 0.5, 0.02, 0.2, "Call")

d1:  -2.0517051739834145
d2:  -2.193126530220724
Call option price:  0.1096559283068772


In [7]:
#Run function
option_price(150, 110, 0.5, 0.02, 0.2, "Put")

d1:  -2.0517051739834145
d2:  -2.193126530220724
Put option price:  38.6171309906821


## Limitations of the model

Whilst in theory the model seems to work efficiently, there are many assumptions that don't hold in practice:

1. The model assumes that the asset follows a lognormal distribution, which means that stock prices are continuous and follow a normal distribution. This assumption does not hold true in reality and many assets are subject to sudden changes and do not necessarily follow a normal distribution.
2. The model assumes that the option can be exercised at any time up to the expiration date, which is known as a European-style option. In reality, there are other types of options, such as American-style options, which can be exercised at any time up to the expiration date.
3. The model assumes that the risk free rate is constant over the lifetime of the option, when in reality there can be changes over time.
4. The model assumes no transaction costs or frictions, when in reality there can be fees or commissions involved. 

## Implied volatility

Implied volatility refers to a metric that captures the market’s view of the likelihood of changes given a security’s price. Investors can use implied volatility to predict future moves and supply and demand, and employ it to price options contracts. 
Implied volatility is the market’s forecast of a likely movement in a security’s price. It is a metric used by investors to estimate future fluctuations (volatility) of a security’s price based based on certain predictive factors. 

When applied to the stock market, implied volatility generally increases in bearish markets, where investors believe markets will decline over time and vice versa when the market is bullish. Implied volatility doesn’t predict the direction in which the price change will proceed, but it implies the size of the price swing. 

### Factors affecting implied volatility

Supply and demand are major factors for implied volatility, when an asset is in high demand, the price tends to rise and so does the implied volatility, which leads to a higher option price due to the risky nature of the option. The opposite is also true.
Another premium influencing factor is the time value of the option, or the amount of time to expiration. A short dated option often results is low implied volatility, whereas a long dated option tends to result in high implied volatility.  

### Define the parameters and functions

In [18]:
# Define Parameters
S = 115
C = 110
t = 1
r = 0.01

In [19]:
def d(S, C, t, r, sigma):
    d1 = (np.log(C/S) + (r + sigma**2/2)*t) / (sigma * np.sqrt(t)) # Expected return adjusted for the risk
    d2 = d1 - (sigma * np.sqrt(t)) # Takes into account the standard deviation of the returns
    return d1, d2

def call_price(d1, d2, S, C, t, r, sigma):
    call_price = (C * norm.cdf(d1)) - (S * np.exp(-r*t) * norm.cdf(d2))
    return call_price

### Newton-Raphson method for finding implied volatility

The Newton-Raphson method works as follows:
1. We start by defining some variables in our model; tol, epsilon, count, max_iter and sigma. The tolerance for the implied volatility is the maximum amount the implied volatility is allowed to be off by. Epsilon is the measure of the difference between the new and old values, which is calculated in the following line of code `epsilon = abs((sigma - orig_vol)/sigma)`. `count` and `max_iter` are variables which allow us to keep track how many times the loop has executed, `max_iter` is important as it acts as a breaking point as to which the code will stop running. Finally, `sigma` represents our starting value for our volatility.
2. The model now starts the while loop.
3. The code runs the `d()` function once in every loop, calculating and returning the `d1` and `d2` variables.
4. The code then runs the `call_price` function, which runs the Black-Scholes equations returning the `price` for the call option.
5. Computes `vega` which is the derivative of the price with respect to the change in volatility.
6. Updates the value of `sigma`.
7. Computes the new value of `epsilon`, which is the difference between the `orig_sigma` and `sigma`.
8. When either the epsilon value is smaller than the tolerance, or the maximum number of iterations the code stops running. If the epsilon value is smaller than the tolerance, the volatility and number of iterations is printed. Otherwise the breaking count is printed.

In [20]:
# Define tolerance, epsilon, count and max iterations
tol = 1e-3
epsilon = 1

count = 0
max_iter = 1000

sigma = 0.5

while epsilon > tol:
    count += 1
    if count >= max_iter:
        print('Breaking count:  ', count)
        break;
    
    orig_sigma = sigma
    
    d1, d2 = d(S, C, t, r, sigma) # Calculates d1 and d2
    price = call_price(d1, d2, S, C, t, r, sigma) # Calculates call price
    
    vega = C * norm.pdf(d1) * np.sqrt(t) # Derivative of price with respect to volatility
    sigma = -price / vega + sigma # Updates the value of sigma
    
    epsilon = abs((sigma - orig_sigma)/sigma) # Calculates differenec between orig_sigma and sigma

print('Volatility: ', sigma)
print('Number of iterations: ', count, 'iterations')

Volatility:  0.001089194827175944
Number of iterations:  505 iterations


### Advantages and disadvantages of using implied volatility

Implied volatility measure has several advantages and disadvantages.

Advantages:
* Provides a metric that can be used to quantify the market expectations surrounding an asset.
* Can be used with a wide range of different option pricing models.
* Provides a way to compare volatility expectations in options with different strike prices and times to expiration.

Disadvantages:
* Can be heavily influenced by supply and demand factors in the markets.
* Assumes that markets can price option efficiently and accurately, in reality that is sometimes not the case.
* Implied volatility may not a good indicator of future volatility if the asset's historical volatility is not a good predictor of future volatility.